Siniflandirma ve regresyon icin kullanilabilir
Robust(dayanikli) bir regresyon modelleme teknigidir

Amac , bir marjin aralgiina maksimum noktayi en kucuk hata ile alabilecek sekilde dogru ya da egriyi belirlemektir

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import scale, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

from warnings import filterwarnings
filterwarnings('ignore')

### Model & Tahmin

In [9]:
df = pd.read_csv('Hitters.csv')
df = df.dropna()
dms= pd.get_dummies(df[['League','Division','NewLeague']]) #kategorik degiskenleri dummy degiskenlere donusturduk
y=df["Salary"]
X_ = df.drop(['Salary','League','Division','NewLeague'],axis=1).astype('float64')# bu 4 sutun kaldirir kalan sutunlari da float64 veri turune cevirir
X=pd.concat([X_,dms[['League_N','Division_W','NewLeague_N']]],axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [10]:
svr_model = SVR(kernel="linear").fit(X_train,y_train)

In [11]:
svr_model

SVR(kernel='linear')

In [14]:
svr_model.predict(X_train)[0:5]

array([219.32624795, 702.43038724, 623.20561122, 153.77539402,
       463.15190748])

In [15]:
svr_model.predict(X_test)[0:5]

array([679.14756648, 633.72885336, 925.68636794, 270.28467553,
       530.26651616])

In [16]:
svr_model.intercept_

array([-80.15195923])

In [17]:
svr_model.coef_

array([[ -1.21839036,   6.0960298 ,  -3.67574544,   0.14217078,
          0.51435925,   1.28388991,  12.55922517,  -0.08693754,
          0.46597181,   2.98259947,   0.5294452 ,  -0.79820801,
         -0.16015542,   0.30872799,   0.28842351,  -1.79560062,
          6.41868986, -10.74313786,   1.33374319]])

In [18]:
#test 
y_pred = svr_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

370.04084355099735

### Model Tuning

In [20]:
svr_params = { "C" : [0.1,0.5,1,3]}

In [29]:
svr_cv_model = GridSearchCV(svr_model,svr_params,cv=5).fit(X_train,y_train)

In [31]:
svr_cv_model.best_params_

{'C': 0.5}

In [34]:
svr_cv_model = GridSearchCV(svr_model,svr_params,cv=5,verbose = 2 ,n_jobs=-1).fit(X_train,y_train)
#verbose, GridSearchCV işlemi sırasında ekrana ne kadar bilgi yazılacağını kontrol eder.
#verbose = 0 hic cikti vermez
#verbose =1 her cv fold icin kisaca ne yaptigini yazar
#verbose = 2 her parametre kombinasyonu icin ve her foldda ne yaptigini ayrinitili yazar

Fitting 5 folds for each of 4 candidates, totalling 20 fits


In [38]:
svr_tuned = SVR(kernel="linear",C=0.5).fit(X_train,y_train)

In [40]:
y_pred = svr_tuned.predict(X_test)

In [42]:
np.sqrt(mean_squared_error(y_test,y_pred))

367.98747185931734